In [1]:
from symmer.symplectic import PauliwordOp
from symmer.projection import (
    S3_projection, QubitTapering, CS_VQE_LW, CS_VQE,
    StabilizerIdentification, ObservableBiasing, stabilizer_walk
)
from symmer.symplectic import *
from symmer.utils import exact_gs_energy, gf2_gaus_elim, gf2_basis_for_gf2_rref
from symmer.chem import geometry_from_pubchem, MoleculeBuilder, get_T2_mp2
from matplotlib import pyplot as plt
from functools import reduce
from openfermion import jordan_wigner
from symmer.utils import QubitOperator_to_dict
import json
#import sys
#sys.path.append( '../../../CS-VQE' )
#from utils.cs_vqe_tools_original import *

In [2]:
with open('notebooks/data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)
with open('notebooks/data/score_data.json', 'r') as infile:
    scoring_data = json.load(infile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiOH_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'NaH_STO-3G_SINGLET', 'LiH_3-21G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'HF_3-21G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2-_STO-3G_SINGLET', 'BH3_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'NH4+_STO-3G_SINGLET', 'MgH2_STO-3G_SINGLET', 'Mg_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3NHCH3_STO-3G_SINGLET', 'CH3CH2NH2_STO-

In [3]:
molecules = [
    'Be_STO-3G_SINGLET',
    'HF_STO-3G_SINGLET',
    'LiH_STO-3G_SINGLET',
    'BeH2_STO-3G_SINGLET',
    'H2O_STO-3G_SINGLET',
    'NH3_STO-3G_SINGLET',
    'CH4_STO-3G_SINGLET',
    'HCl_STO-3G_SINGLET',
    'F2_STO-3G_SINGLET',
    'H2S_STO-3G_SINGLET',
    'LiOH_STO-3G_SINGLET',
    'HF_3-21G_SINGLET',
    'LiH_3-21G_SINGLET',
    'CO_STO-3G_SINGLET',
    'NaH_STO-3G_SINGLET',
    'NH4+_STO-3G_SINGLET',
    'MgH2_STO-3G_SINGLET',
    'OH-_STO-3G_SINGLET',
    'Mg_STO-3G_SINGLET',
    'HCN_STO-3G_SINGLET'
]

def get_score_vs_error(cs_vqe_obj, S, max_sim_q=19):
    if S is not None:
        stab_score  = cs_vqe_obj.basis_score(S)
        ham_cs = cs_vqe_obj.project_onto_subspace(S, enforce_clique_operator=False)
        n_qubits = ham_cs.n_qubits
        if i<=max_sim_q:
            matrix = ham_cs.to_sparse_matrix
            cs_vqe_energy = exact_gs_energy(matrix)[0]
            #error  = energy-molecule.fci_energy
        else:
            cs_vqe_energy = None
    else:
        stab_score = None
        cs_vqe_energy = None
        
    return stab_score, cs_vqe_energy

In [6]:
import sys
sys.path.append( '../CS-VQE' )
from utils.cs_vqe_tools_original import *

In [ ]:
original_results = {}

for speciesname in molecules:
    print(speciesname)
    # build the molecule
    mol_data = molecule_geometries[speciesname]
    atoms  = mol_data['atoms']
    coords = mol_data['coords']
    charge = mol_data['charge']
    basis = mol_data['basis']
    geometry = list(zip(atoms, coords))
    molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

    # taper the Hamiltonian
    taper_hamiltonian = QubitTapering(molecule.H_q)
    hf_array = molecule.H_fermion.hf_comp_basis_state
    taper_hamiltonian.stabilizers.update_sector(hf_array)
    ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
    ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
    hf_tapered = taper_hamiltonian.tapered_ref_state

    H = ham_tap.to_dictionary
    H_noncon = greedy_dfs(H, cutoff=10)[-1]
    H_noncon = {P:H[P] for P in H_noncon}
    nc_legacy, energies_legacy, errors_legacy, order_legacy = csvqe_approximations_heuristic(
        H, H_noncon, ham_tap.n_qubits, molecule.fci_energy)
    
    print(np.array(energies_legacy)-molecule.fci_energy)
    
    data = {'cs_vqe_nrg':   energies_legacy,
            'cs_vqe_errors':errors_legacy,
            'noncon_energy':nc_legacy,
            'fci_energy':   molecule.fci_energy}
            
    original_results[speciesname] = data
    
    with open('notebooks/data/original_results_qlm.json', 'w') as outfile:
        json.dump(original_results, outfile)


Be_STO-3G_SINGLET
Molecule geometry:
Be	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -14.351880476202012
MP2 energy:  -14.376238850841505
CCSD energy: -14.403650751752428
FCI energy:  -14.403655108067667


Number of qubits: 10
[ 5.17746319e-02  3.08642042e-02  1.42160388e-02  3.26505984e-04
  1.75978902e-04 -1.59872116e-14]
HF_STO-3G_SINGLET
Molecule geometry:
H	0.0	0.0	0.099457
F	0.0	0.0	-0.895109

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -98.57101106797647
MP2 energy:  -98.59198165670847
CCSD energy: -98.60330177389832
FCI energy:  -98.60330177725879


Number of qubits: 12
[ 3.22907093e-02  3.22907093e-02  6.81210832e-03  4.70452199e-03
  8.08666615e-04  4.01734348e-04  1.99122551e-05  1.36247735e-05
 -7.10542736e-14]
LiH_STO-3G_SINGLET
Molecule geometry:
Li	0.0	0.0	0.3868
H	0.0	0.0	-1.1605

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -7.863115321486988
MP2 energy:  -7.87562527

In [6]:
for speciesname in ['HF_3-21G_SINGLET','LiH_3-21G_SINGLET']: #molecules:
    print(speciesname)
    # build the molecule
    mol_data = molecule_geometries[speciesname]
    atoms  = mol_data['atoms']
    coords = mol_data['coords']
    charge = mol_data['charge']
    basis = mol_data['basis']
    geometry = list(zip(atoms, coords))
    molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

    MP2 = PauliwordOp(
        QubitOperator_to_dict(
            jordan_wigner(
                get_T2_mp2(
                    molecule.pyscf_obj.pyscf_mp2.t2)
            ), 
            molecule.n_qubits)
    ).sort()

    # taper the Hamiltonian
    taper_hamiltonian = QubitTapering(molecule.H_q)
    hf_array = molecule.H_fermion.hf_comp_basis_state
    taper_hamiltonian.stabilizers.update_sector(hf_array)
    ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
    ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
    mp2_tap = taper_hamiltonian.taper_it(aux_operator=MP2, ref_state=hf_array)
    hf_tapered = taper_hamiltonian.tapered_ref_state

    # initiate stabilizer identification classes
    Hc_stabilizers = StabilizerIdentification(ham_tap)
    CC_stabilizers = StabilizerIdentification(ucc_tap)
    MP_stabilizers = StabilizerIdentification(mp2_tap)
    HL_gap = np.where(hf_tapered==0)[0][0]-.5
    ham_biasing = ObservableBiasing(base_operator=ham_tap, HOMO_LUMO_gap=HL_gap)

    # build CS-VQE model
    cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')

    data = {
        'stab_score_Hc':[],
        'stab_score_CC':[],
        'stab_score_MP':[],
        'stab_score_HL':[],
        'cs_vqe_nrg_Hc':[],
        'cs_vqe_nrg_CC':[],
        'cs_vqe_nrg_MP':[],
        'cs_vqe_nrg_HL':[]
    }

    for i in range(1, ham_tap.n_qubits):
        print(i)
        #data['n_sim_qubits'].append(i)
        # by Hc
        try:
            S_Hc = Hc_stabilizers.symmetry_basis_by_subspace_dimension(i)
        except:
            S_Hc = None
        stab_score_Hc, energy_Hc = get_score_vs_error(cs_vqe, S_Hc)
        data['stab_score_Hc'].append([i, stab_score_Hc])
        data['cs_vqe_nrg_Hc'].append([i, energy_Hc])

        # by CC
        try:
            S_CC = CC_stabilizers.symmetry_basis_by_subspace_dimension(i)
        except:
            S_CC = None
        stab_score_CC, energy_CC = get_score_vs_error(cs_vqe, S_CC)
        data['stab_score_CC'].append([i, stab_score_CC])
        data['cs_vqe_nrg_CC'].append([i, energy_CC])

        # by MP
        try:
            S_MP = MP_stabilizers.symmetry_basis_by_subspace_dimension(i)
        except:
            S_MP = None
        stab_score_MP, energy_MP = get_score_vs_error(cs_vqe, S_MP)
        data['stab_score_MP'].append([i, stab_score_MP])
        data['cs_vqe_nrg_MP'].append([i, energy_MP])

        # by HOMO-LUMO biasing
        try:
            S_HL = stabilizer_walk(
                    n_sim_qubits=i, 
                    biasing_operator=ham_biasing, 
                    cs_vqe_object=cs_vqe,
                    #tapering_object=taper_hamiltonian,
                    #reference_state=hf_array
                )
        except:
            S_HL = None
        stab_score_HL, energy_HL = get_score_vs_error(cs_vqe, S_HL)
        data['stab_score_HL'].append([i, stab_score_HL])
        data['cs_vqe_nrg_HL'].append([i, energy_HL])

    stabilizer_identification_comparison[speciesname] = {
            'n_qubits_pre_taper': int(molecule.n_qubits),
            'n_qubits_post_taper': int(ham_tap.n_qubits),
            'hf_energy': molecule.hf_energy,
            'mp2_energy': molecule.mp2_energy,
            'ccsd_energy': molecule.ccsd_energy,
            'fci_energy': molecule.fci_energy,
            'noncon_energy': cs_vqe.noncontextual_energy,
            'data':data
        }

    with open('notebooks/data/stabilizer_identification_comparison_qlm.json', 'w') as outfile:
        json.dump(stabilizer_identification_comparison, outfile)


HF_3-21G_SINGLET
Molecule geometry:
F	0.0	0.0	0.093761
H	0.0	0.0	-0.84385

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -99.46021892473368
MP2 energy:  -99.58344905881171
CCSD energy: -99.5870629471661
FCI energy:  -99.58791845765413


Number of qubits: 22
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
LiH_3-21G_SINGLET
Molecule geometry:
Li	0.0	0.0	0.409973
H	0.0	0.0	-1.22992

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -7.929842624886332
MP2 energy:  -7.942716950764573
CCSD energy: -7.949237304438306
FCI energy:  -7.949249470779712


Number of qubits: 22
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [32]:
stabilizer_identification_comparison

{'F2_STO-3G_SINGLET': {'n_qubits_pre_taper': 20,
  'n_qubits_post_taper': 16,
  'hf_energy': -195.97354107487368,
  'mp2_energy': -196.0227691305834,
  'ccsd_energy': -196.05016033964972,
  'fci_energy': -196.05016034219025,
  'noncon_energy': -195.97354107487342,
  'data': {'stab_score_Hc': [[1, 0.0],
    [2, 0.03422830605267146],
    [3, 0.12303021146783791],
    [4, 0.12441021367839418],
    [5, 0.1269304772384124],
    [6, 0.12797020402022588],
    [7, 0.17130144589027296],
    [8, 0.2455683285497565],
    [9, 0.8299891046333685],
    [10, 0.8299891046333685],
    [11, 0.8299891046333685],
    [12, 0.8299891046333685],
    [13, 0.8299891046333685],
    [14, 0.914994552316684],
    [15, 1.0]],
   'stab_score_CC': [[1, 0.5101538934241284],
    [2, 0.5844088473070999],
    [3, 0.7009553162783051],
    [4, 0.7674148483000798],
    [5, 0.8338743803218532],
    [6, 0.8884479796843483],
    [7, 0.9569045917896913],
    [8, 0.9754505074512332],
    [9, 0.9939964231127747],
    [10, 0.99537

In [16]:
mol_data = molecule_geometries[speciesname]
atoms  = mol_data['atoms']
coords = mol_data['coords']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=0, basis='STO-3G', spin=0, run_fci=False, print_info=False)

MP2 = PauliwordOp(
    QubitOperator_to_dict(
        jordan_wigner(
            get_T2_mp2(
                molecule.pyscf_obj.pyscf_mp2.t2)
        ), 
        molecule.n_qubits)
).sort()

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
mp2_tap = taper_hamiltonian.taper_it(aux_operator=MP2, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# initiate stabilizer identification classes
Hc_stabilizers = StabilizerIdentification(ham_tap)
CC_stabilizers = StabilizerIdentification(ucc_tap)
MP_stabilizers = StabilizerIdentification(mp2_tap)
#HL_gap = np.where(hf_tapered==0)[0][0]-.5
#ham_biasing = ObservableBiasing(base_operator=ham_tap, HOMO_LUMO_gap=HL_gap)

# build CS-VQE model
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')

data = {'n_sim_qubits':[],
       'stab_score_Hc':[],
       'stab_score_CC':[],
       'stab_score_MP':[],
       'stab_score_HL':[],
       'cs_vqe_nrg_Hc':[],
       'cs_vqe_nrg_CC':[],
       'cs_vqe_nrg_MP':[],
       'cs_vqe_nrg_HL':[]}



In [21]:
S_Hc = Hc_stabilizers.symmetry_basis_by_subspace_dimension(10)
stab_score_Hc, energy_Hc = get_score_vs_error(cs_vqe, S_Hc)
data['stab_score_Hc'].append(stab_score_Hc)
data['cs_vqe_nrg_Hc'].append(energy_Hc)

In [22]:
data

{'n_sim_qubits': [],
 'stab_score_Hc': [0.11178453494439002,
  0.11034988048678479,
  0.24716190437798183],
 'stab_score_CC': [],
 'stab_score_MP': [],
 'stab_score_HL': [],
 'cs_vqe_nrg_Hc': [-394.32061994051844,
  -394.32061419308235,
  -394.3260674223234],
 'cs_vqe_nrg_CC': [],
 'cs_vqe_nrg_MP': [],
 'cs_vqe_nrg_HL': []}